# nbgrader

> This module process the nbgrader generated csv grades files and the output directories, and use canvas api to post grade to students, with late penalty and the messages.

In [1]:
#| default_exp nbgrader

In [2]:
#| hide
from nbdev.showdoc import *

In [122]:
#| export
import canvasapi
from canvasapi import Canvas
import numpy as np
import pandas as pd
import json
from datetime import datetime
import yaml

In [125]:
with open("late_exception.yaml", "r") as f:
    exception = yaml.safe_load(f)
exception

{'grader-cogs108-02': {'late_days': 7, 'reasons': 'sickness'},
 'student2': {'late_days': 10, 'reasons': 'family issue'}}

In [4]:
#| export
#| hide
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [127]:
#| export
class nbgrader_grade:
    def __init__(self,
                 credentials_fp="", # credential file path. [Template of the credentials.json](https://github.com/FleischerResearchLab/CanvasGroupy/blob/main/nbs/credentials.json)
                 late_exception_fp="", # late exception yaml file path. [Template of the late_exception.yaml](https://github.com/scott-yj-yang/GradescopeLatePenalty/tree/main/nbs/api/late_exception.yaml)
                 API_URL="https://canvas.ucsd.edu", # the domain name of canvas
                 course_id="", # Course ID, can be found in the course url
                 assignment_id=-1, # assignment id, can be found in the canvas assignment url
                 grades_fp="", # nbgrader csv grades exports file path 
                 verbosity=0 # Controls the verbosity: 0 = Silent, 1 = print all messages
                ):
        "Initialize Canvas Group within a Group Set and its appropriate memberships"
        self.API_URL = API_URL
        self.canvas = None
        self.course = None
        self.users = None
        self.email_to_canvas_id = None
        self.canvas_id_to_email = None
        self.API_KEY = None
        self.verbosity = verbosity
        self.assignment = None
        self.grades = None
        self.late_exception = dict()
        self.grades_by_assignment = dict()
        self.late_days_by_assignment = dict()
        
        # initialize by the input parameter
        if credentials_fp != "":
            self.auth_canvas(credentials_fp)
        if course_id != "":
            self.set_course(course_id)
        if assignment_id != -1:
            self.link_assignment(assignment_id)
        if gradescope_fp != "":
            self.load_gradescope_csv(gradescope_fp)
        if late_exception_fp !="":
            self.load_late_exception(late_exception_fp)
        
    def auth_canvas(self,
                    credentials_fp: str # the Authenticator key generated from canvas
                   ):
        "Authorize the canvas module with API_KEY"
        with open(credentials_fp, "r") as f:
            credentials = json.load(f)
        self.API_KEY = credentials["Canvas Token"]
        self.GITHUB_TOKEN = credentials["GitHub Token"]
        self.canvas = Canvas(self.API_URL, self.API_KEY)
        # test authorization
        _ = self.canvas.get_activity_stream_summary()
        if self.verbosity != 0:
            print(f"{bcolors.OKGREEN}Authorization Successful!{bcolors.ENDC}")
        
    def set_course(self, 
                   course_id: int # the course id of the target course
                  ):
        "Set the target course by the course ID"
        self.course = self.canvas.get_course(course_id)
        if self.verbosity != 0:
            print(f"Course Set: {bcolors.OKGREEN} {self.course.name} {bcolors.ENDC}")
            print(f"Getting List of Users... This might take a while...")
        self.users = list(self.course.get_users())
        if self.verbosity != 0:
            print(f"Users Fetch Complete! The course has {bcolors.OKBLUE}{len(self.users)}{bcolors.ENDC} users.")
        self.email_to_canvas_id = {}
        self.canvas_id_to_email = {}
        for u in self.users:
            try:
                self.email_to_canvas_id[u.email.split("@")[0]] = u.id
                self.canvas_id_to_email[u.id] = u.email.split("@")[0]
            except Exception:
                if self.verbosity != 0:
                    print(f"{bcolors.WARNING}Failed to Parse email and id"
                          f" for {bcolors.UNDERLINE}{u.short_name}{bcolors.ENDC}{bcolors.ENDC}")

    def link_assignment(self,
                        assignment_id: int # assignment id, found at the url of assignmnet tab
                       ) -> canvasapi.assignment.Assignment: # target assignment
        "Link the target assignment on canvas"
        assignment = self.course.get_assignment(assignment_id)
        if self.verbosity != 0:
            print(f"Assignment {bcolors.OKGREEN+assignment.name+bcolors.ENDC} Link!")
        self.assignment = assignment
        return assignment
                    
    def load_grades_csv(self,
                        csv_pf:str # csv file path 
                       ):
        "Load nbgrader exported csv file"
        self.grades = pd.read_csv(csv_pf)
    
    def load_late_exception(self,
                            yaml_fp:str # yaml file path stores exception student cases
                           ):
        "Load Late Exception File"
        with open(yaml_fp, "r") as f:
            self.late_exception = yaml.safe_load(f)
        
    def _parse_assignments(self):
        "Parse all assignment by assignment name. And calculate late days used."
        if self.grades == None:
            raise ValueError("grades has not been loaded. Please loaded via self.load_grades_csv")
        assignments = self.grades["assignment"].unique()
        # I am just lazy :-)
        df = self.grades
        for assignment in assignments:
            A = df[df["assignment"] == assignment]
            # filter those who submitted
            A = A[~A["timestamp"].isna()].copy()
            # remove the redundant /
            A["student_id"] = A["student_id"].str.split("/").str[0]
            A = D1.set_index("student_id")
            slip_day_used = self._calculate_late_days(A)
            A["slip_day_used"] = slip_day_used
            # store the parsed result
            self.grades_by_assignment[assignment] = A
            self.late_days_by_assignment[assignment] = A["slip_day_used"]
    
    def _calculate_late_days(self,
                             df:pd.DataFrame # dataframe of a specific assignment
                            )-> pd.Series: # late days
        # parse the timestamp
        duedate_format = "%Y-%m-%d %H:%M:%S"
        timestamp_format = "%Y-%m-%d %H:%M:%S.%f"
        df["duedate"] = df["duedate"].apply(lambda x: datetime.strptime(x, duedate_format))
        df["timestamp"] = df["timestamp"].apply(lambda x: datetime.strptime(x, timestamp_format))
        late_time_delta = (df["timestamp"] - df["duedate"])
        # calculate late days, use ReLU
        slip_day_used = late_time_delta.apply(lambda x: np.max([np.ceil(x.total_seconds()/60/60/24), 0]))
        return slip_day_used
    
    def get_late_days(self,
                      target_assignment:str, # target assignment name. Must in the column of nbgrader assignment csv 
                      student_id:str # student id
                     ) -> int: # late days of the target assignment
        "Calculate the late day of students submission of the target assignment"
        try:
            late_day = self.late_days_by_assignment[target_assignment][student_id]
        except KeyError:
            if self.verbosity != 1:
                print(f"Student {bcolors.WARNING+student_id+bcolors.ENDC} did "
                      f"not submit {bcolors.WARNING+target_assignment+bcolors.ENDC}")
            late_day = 0
        return late_day
        
    def calculate_credit_balance(self,
                                 passed_assignments:[str], # list of passed assignments name. Must in the column of `assignment`.
                                 student_id:str, # target student
                                 default_credit = 5 # default total number of allowed late days
                                ) -> int: # late credit balance of the target student
        "Calculate the balance of late hours from the nbgrader file"
        # if student is in the late exception, use the new number
        if student_id in self.late_days_by_assignment:
            default_credit = self.late_days_by_assignment[student_id]["allowed_late_days"]
        for passed in passed_assignments:
            default_credit -= self.get_late_days(passed, student_id)
        return default_credit
    
    def _post_grade(self,
                   student_id: int, # canvas student id of student. found in self.email_to_canvas_id
                   grade: float, # grade of that assignment
                   text_comment="", # text comment of the submission. Can feed
                  ) -> canvasapi.submission.Submission: # created submission
        "Post grade and comment to canvas to the target assignment"
        submission = self.assignment.get_submission(student_id)
        edited = submission.edit(
            submission={
                'posted_grade': grade
            }, comment={
                'text_comment': text_comment
            }
        )
        if self.verbosity != 0:
            print(f"Grade for {bcolors.OKGREEN+email+bcolors.ENDC} Posted!")
        return edited
    
    def post_to_canvas(self,
                       target_assignment:str, # target assignment name to grab the late time. Must in the column of nbgrader assignment csv 
                       passed_assignments:[str], # list of passed assignment. Must in the column of nbgrader assignment csv
                       post=True, # for testing purposes. Can hault the post operation
                      ):
        "Post grade to canvas with late penalty."
        if self.grades is None:
            raise ValueError("Nbgrader CSV has not been loaded. Please set it via self.load_grades_csv")
        for studend_id, row in self.grades_by_assignment[target_assignment].iterrows():
            # fetch useful information
            balance = self.calculate_credit_balance(passed_assignments, studend_id)
            late_days = self.get_late_days(target_assignment, studend_id)
            score = row["raw_score"]
            # build message for each student
            message = f"{target_assignment}: \n"
            if late_days > 0:
                # means late submission. Check remaining slip day
                message += f"Late Submission: {int(late_days)} Days Late\n"
                if balance - late_days < 0:
                    message += "Insufficient Slip Credit. 25% late penalty applied\n"
                    score = round(score * 0.75, 4)
                else:
                    message += "Slip Credit Used. No late penalty applied\n"
            else:
                message += "Submitted intime\n"
            balance_after = balance - late_days
            message += f"Remaining Slip Credit: {int(balance_after)} Days"
            try:
                if post:
                    canvas_student_id = grade.email_to_canvas_id[studend_id]
                    grade._post_grade(grade=score, student_id=canvas_student_id, text_comment=message)
                else:
                    print(f"{bcolors.WARNING}Post Disabled{bcolors.ENDC}\n"
                          f"The message is: \n{bcolors.OKGREEN+message+bcolors.ENDC}"
                         )
            except Exception as e:
                print(f"Studnet: {bcolors.WARNING+email+bcolors.ENDC} Not found on canvas. \n"
                      f"Maybe Testing Account or Dropped Student")
                print(e)
                pass
        

# Initialization 

In [128]:
show_doc(nbgrader_grade)

---

### nbgrader_grade

>      nbgrader_grade (credentials_fp='', late_exception_fp='',
>                      API_URL='https://canvas.ucsd.edu', course_id='',
>                      assignment_id=-1, grades_fp='', verbosity=0)

Initialize Canvas Group within a Group Set and its appropriate memberships

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| credentials_fp | str |  | credential file path. [Template of the credentials.json](https://github.com/FleischerResearchLab/CanvasGroupy/blob/main/nbs/credentials.json) |
| late_exception_fp | str |  | late exception yaml file path. [Template of the late_exception.yaml](https://github.com/scott-yj-yang/GradescopeLatePenalty/tree/main/nbs/api/late_exception.yaml) |
| API_URL | str | https://canvas.ucsd.edu | the domain name of canvas |
| course_id | str |  | Course ID, can be found in the course url |
| assignment_id | int | -1 | assignment id, can be found in the canvas assignment url |
| grades_fp | str |  | nbgrader csv grades exports file path |
| verbosity | int | 0 | Controls the verbosity: 0 = Silent, 1 = print all messages |

In [129]:
grade = nbgrader_grade("../../../credentials.json",
                       late_exception_fp="late_exception.yaml",
                       course_id=45059,
                       assignment_id=641795,
                       grades_fp="../../../gradescope.csv",
                       verbosity=1,
                      )

FileNotFoundError: [Errno 2] No such file or directory: '../../../credentials.json'

In [ ]:
show_doc(process_grade.post_to_canvas)

---

[source](https://github.com/scott-yj-yang/GradescopeLatePenalty/blob/main/GradescopeLatePenalty/process_grade.py#L164){target="_blank" style="float:right; font-size:smaller"}

### process_grade.post_to_canvas

>      process_grade.post_to_canvas (target_assignment:str,
>                                    passed_assignments:[<class'str'>],
>                                    components:[<class'str'>], post=True)

Post grade to canvas with late penalty.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| target_assignment | str |  | target assignment name to grab the late time. Must in the column of gradescope csv |
| passed_assignments | [<class 'str'>] |  | list of passed assignment. Must in the column of gradescope csv |
| components | [<class 'str'>] |  | components of a single assignment. Must in the column of gradescope csv |
| post | bool | True | for testing purposes. Can hault the post operation |

In [ ]:
show_doc(process_grade.calculate_total_score)

---

[source](https://github.com/scott-yj-yang/GradescopeLatePenalty/blob/main/GradescopeLatePenalty/process_grade.py#L137){target="_blank" style="float:right; font-size:smaller"}

### process_grade.calculate_total_score

>      process_grade.calculate_total_score (components:[<class'str'>])

Calculate the total score of an assignment

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| components | [<class 'str'>] | components of a single assignment. Must in the column of gradescope csv |
| **Returns** | **Series** |  |

In [ ]:
show_doc(process_grade.calculate_credit_balance)

---

[source](https://github.com/scott-yj-yang/GradescopeLatePenalty/blob/main/GradescopeLatePenalty/process_grade.py#L126){target="_blank" style="float:right; font-size:smaller"}

### process_grade.calculate_credit_balance

>      process_grade.calculate_credit_balance
>                                              (passed_assignments:[<class'str'>
>                                              ], total_credit=120)

Calculate the balance of late hours from the gradescope file

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| passed_assignments | [<class 'str'>] |  | list of passed assignment. Must in the column of gradescope csv |
| total_credit | int | 120 | total number of allowed late hours |
| **Returns** | **dict** |  | **{email: credit balance} late credit balance of each students** |

In [ ]:
show_doc(process_grade.calculate_late_hour)

---

[source](https://github.com/scott-yj-yang/GradescopeLatePenalty/blob/main/GradescopeLatePenalty/process_grade.py#L113){target="_blank" style="float:right; font-size:smaller"}

### process_grade.calculate_late_hour

>      process_grade.calculate_late_hour (target_assignment:str)

Calculate the late hours of each submission of the target assignment

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| target_assignment | str | target assignment name. Must in the column of gradescope csv |
| **Returns** | **Series** | **late hours of the target assignment** |

In [ ]:
show_doc(process_grade._post_grade)

---

[source](https://github.com/scott-yj-yang/GradescopeLatePenalty/blob/main/GradescopeLatePenalty/process_grade.py#L146){target="_blank" style="float:right; font-size:smaller"}

### process_grade._post_grade

>      process_grade._post_grade (student_id:int, grade:float, text_comment='')

Post grade and comment to canvas to the target assignment

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| student_id | int |  | canvas student id of student. found in self.email_to_canvas_id |
| grade | float |  | grade of that assignment |
| text_comment | str |  | text comment of the submission. Can feed |
| **Returns** | **Submission** |  | **created submission** |

In [ ]:
#| hide
# grade.calculate_credit_balance(["Assignment 1"])

In [ ]:
#| hide
# grade.calculate_total_score(["Assignment 1", "Assignment 1 - Manual Grading"])

In [ ]:
#| hide
# grade.calculate_late_hour("Assignment 1")

In [ ]:
#| hide
# example

# assignment = grade.course.get_assignment(641795)
# student_id = grade.email_to_canvas_id["grader-cogs118a-01"]
# submission = assignment.get_submission(student_id)
# submission.edit(
#     submission={
#         'posted_grade': 0
#     }, comment={
#         'text_comment': "You are the best \n but you did submit late \n so I have to give you 0"
#     }
# )

In [ ]:
#| hide
# # This is forbidden... Ask for the privilage?
# assignment.submit(
#     submission={
#         "submission_type": "oneline_upload",
#         "user_id": student_id,
#         "submitted_at": submission_time
#     }
# )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()